# Make bookkeeping For Bias


- work with Weakly_2022_26
- use jupyter kernel LSST

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/06/29


In [1]:
! eups list -s | grep LOCAL

eups                  LOCAL:/cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/w_2022_26/conda/envs/lsst-scipipe-4.0.1-ext/eups 	setup


In [2]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib


   g0b29ad24fb+e8b8cae3ca 	w_latest w_2022_26 current setup


In [3]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits

In [4]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [5]:
import lsst.daf.butler as dafButler

In [6]:
repo = '/sps/lsst/groups/auxtel/softs/shared/auxteldm_gen3/data/butler.yaml'
butler = dafButler.Butler(repo)
registry = butler.registry

In [7]:
collection = "u/dagoret/first_test2"
datasetRefs1 = registry.queryDatasets(datasetType='postISRCCD', collections=collection, where= "instrument='LATISS'")
datasetRefs2 = registry.queryDatasets(datasetType='calexp', collections=collection, where= "instrument='LATISS'")

## List of Exposures

In [8]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs','group_name' ,'seq_num', 'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle','postISRCCD','calexp'])

In [9]:
df_exposure.astype({"id":str,
                    "obs_id":str,
                    "day_obs": str,
                    "group_name":str,
                    "seq_num":'int32',
                    'type':str,
                    "target":str,
                    "filter":str,
                    "zenith_angle":'float',
                    "expos":'float',
                    "ra":'float',
                    "dec":'float',
                    "skyangle":'float',
                    "postISRCCD":bool,
                    "calexp":bool
                   }              
                  )

,id,obs_id,day_obs,group_name,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle,postISRCCD,calexp


In [10]:
#df_exposure = pd.DataFrame( {"id":'int64',
#                             "obs_id":str,
#                             "day_obs": str,
#                             "seq_num":'int32',
#                             "type":str,
#                             "target":str,
#                             "filter":str,
#                             "zenith_angle":'float32',
#                             "expos":'float32',
#                             "ra":'float32',
#                             "dec":'float32',
#                             "skyangle":'float32'})                  

In [11]:
for count, info in enumerate(registry.queryDimensionRecords('exposure')):
    
    flag_postisrccd = False
    flag_calexp = False
    
    #for count, ref in enumerate(datasetRefs1):    
    #    if ref.dataId["exposure"]== info.id:
    #        flag_postisrccd = True
    #        break
    #for count, ref in enumerate(datasetRefs2):    
    #    if ref.dataId["exposure"]== info.id:
    #        flag_calexp = True
    #        break        
            
    
    
    
    df_exposure.loc[count] = [str(info.id), str(info.obs_id), str(info.day_obs), str(info.group_name),int(info.seq_num), str(info.observation_type), str(info.target_name), str(info.physical_filter), info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle,flag_postisrccd,flag_calexp]
    
    if count < 0:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t group_name:          ",info.group_name)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
    

In [12]:
df_exposure.dtypes

id               object
obs_id           object
day_obs          object
group_name       object
seq_num           int64
type             object
target           object
filter           object
zenith_angle     object
expos           float64
ra               object
dec              object
skyangle         object
postISRCCD         bool
calexp             bool
dtype: object

In [13]:
df_exposure

,id,obs_id,day_obs,group_name,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle,postISRCCD,calexp
0,2021021600208,AT_O_20210216_000208,20210216,2021-02-17T03:35:45.035,208,science,HD 50896,empty~empty,24.314073,2.0,103.384002,-23.973448,-180.0,False,False
1,2021021600163,AT_O_20210216_000163,20210216,2021-02-17T00:50:15.536,163,science,HD 50896,RG610~empty,14.580611,3.0,103.404029,-23.92939,-208.621712,False,False
2,2021021600070,AT_O_20210216_000070,20210216,2021-02-16T20:39:31.417,70,bias,azel_target,empty~empty,None,0.0,None,None,None,False,False
3,2021021600173,AT_O_20210216_000173,20210216,2021-02-17T01:21:19.215,173,science,HD 50896,RG610~empty,8.82931,2.0,103.395986,-23.927038,-208.621712,False,False
4,2021021600001,AT_O_20210216_000001,20210216,2021-02-16T19:50:21.993,1,bias,azel_target,empty~empty,None,0.0,None,None,None,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19070,2022060900838,AT_O_20220609_000838,20220609,2022-06-10T08:52:06.482,838,science,spec:HD185975,BG40~holo4_003,57.194196,30.0,307.708009,-87.50686,105.70201,False,False
19071,2022060900839,AT_O_20220609_000839,20220609,2022-06-10T08:52:06.482,839,science,spec:HD185975,FELH0600~holo4_003,57.196255,30.0,307.707897,-87.506868,105.701922,False,False
19072,2022060900840,AT_O_20220609_000840,20220609,2022-06-10T08:52:06.482,840,science,spec:HD185975,FELH0600~holo4_003,57.198265,30.0,307.708264,-87.506854,105.701541,False,False
19073,2022060900841,AT_O_20220609_000841,20220609,2022-06-10T08:52:06.482,841,science,spec:HD185975,FELH0600~holo4_003,57.200339,30.0,307.708109,-87.506873,105.701644,False,False


## Selection of Bias

In [14]:
df_bias = df_exposure[df_exposure.type == 'bias']

In [15]:
df_bias.sort_values(by="id",ascending=True,inplace=True)

/tmp/ipykernel_3172/1884773382.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bias.sort_values(by="id",ascending=True,inplace=True)


In [16]:
df_bias.tail(60)

,id,obs_id,day_obs,group_name,seq_num,type,target,filter,zenith_angle,expos,ra,dec,skyangle,postISRCCD,calexp
17491,2022060800005,AT_O_20220608_000005,20220608,2022-06-08T20:55:43.025,5,bias,FlatField position,empty~empty,None,0.0,None,None,None,False,False
17492,2022060800006,AT_O_20220608_000006,20220608,2022-06-08T20:55:47.519,6,bias,FlatField position,empty~empty,None,0.0,None,None,None,False,False
17493,2022060800007,AT_O_20220608_000007,20220608,2022-06-08T20:55:52.019,7,bias,FlatField position,empty~empty,None,0.0,None,None,None,False,False
17494,2022060800008,AT_O_20220608_000008,20220608,2022-06-08T20:55:56.515,8,bias,FlatField position,empty~empty,None,0.0,None,None,None,False,False
17495,2022060800009,AT_O_20220608_000009,20220608,2022-06-08T20:56:01.004,9,bias,FlatField position,empty~empty,None,0.0,None,None,None,False,False
17496,2022060800010,AT_O_20220608_000010,20220608,2022-06-08T20:56:05.491,10,bias,FlatField position,empty~empty,None,0.0,None,None,None,False,False
17497,2022060800011,AT_O_20220608_000011,20220608,2022-06-08T20:56:09.981,11,bias,FlatField position,empty~empty,None,0.0,None,None,None,False,False
17498,2022060800012,AT_O_20220608_000012,20220608,2022-06-08T20:56:14.472,12,bias,FlatField position,empty~empty,None,0.0,None,None,None,False,False
17499,2022060800013,AT_O_20220608_000013,20220608,2022-06-08T20:56:18.964,13,bias,FlatField position,empty~empty,None,0.0,None,None,None,False,False
17500,2022060800014,AT_O_20220608_000014,20220608,2022-06-08T20:56:23.454,14,bias,FlatField position,empty~empty,None,0.0,None,None,None,False,False


# Generate Book Keeping files

In [17]:
writer = pd.ExcelWriter('BookKeepingAuxtelBias_raw.xlsx', engine='xlsxwriter')
current_page = 0

ModuleNotFoundError: No module named 'xlsxwriter'

In [ ]:
ListOfDates = df_bias.day_obs.unique()

In [ ]:
ListOfDates

In [ ]:
for date_sel in ListOfDates:
    
    # get for that date
    df_bias_selected = df_bias[df_bias.day_obs == date_sel ]
    N = len(df_bias_selected)
    
    list_of_filters = df_bias_selected['filter'].unique()
    
    selected_filters = []
    # Select hologram
    for filt in list_of_filters:
        #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
        #flag_sel = (filt.find('holo4') != -1) 
        #flag_sel = (filt.find('ronchi170') != -1)
        flag_sel = True
        if flag_sel:
            selected_filters.append(filt) 
    Nfilt = len(selected_filters)
    # sort by filter name
    if(Nfilt>0):
        selected_filters = np.array(selected_filters)
        df_bias_selected.sort_values(by=['filter','id'],ascending=True,inplace=True)
        df_bias_selected.to_excel(writer, sheet_name=str(date_sel))
        


In [ ]:
writer.save()